In [11]:
import math
import torch
import torch.nn as nn
from d2l import torch as d2l

OSError: /home/zhouquan/anaconda3/envs/DLstudy/lib/python3.10/site-packages/nvidia/cublas/lib/libcublas.so.11: symbol cublasLtHSHMatmulAlgoInit version libcublasLt.so.11 not defined in file libcublasLt.so.11 with link time reference